In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from larcv import larcv
larcv.load_pyutil
from ROOT import TChain

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#FNAME='segmentation_iter_3500.root'
FNAME='segmentation_spweights_iter_4500.root'

ch=TChain("image2d_ssnet_plane2_tree")
ch.AddFile(FNAME)
print ch.GetEntries()

In [ ]:
ASPECT=1.0
XLIM_DEFAULT=None
YLIM_DEFAULT=None

for x in xrange(ch.GetEntries()):
    
    ch.GetEntry(x)
    print 'Entry',x
    image2d_v = ch.image2d_ssnet_plane2_branch.Image2DArray()
    print image2d_v.size()
    fig,ax_arr=plt.subplots(1,5,figsize=(64,16),facecolor='w')
    
    bkgimg    = np.flipud(larcv.as_ndarray(image2d_v[0]).astype(np.float32))*255.
    showerimg = np.flipud(larcv.as_ndarray(image2d_v[1]).astype(np.float32))*255.
    trackimg  = np.flipud(larcv.as_ndarray(image2d_v[2]).astype(np.float32))*255.
    adcimg    = np.flipud(larcv.as_ndarray(image2d_v[3]).astype(np.float32))*1.
    labelimg  = np.flipud(larcv.as_ndarray(image2d_v[4]).astype(np.float32))*80.
    
    nz_pixels=np.where(adcimg>40.0)
    xlim,ylim = (None,None)
    if YLIM_DEFAULT: ylim = YLIM_DEFAULT
    else: ylim = (np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    if XLIM_DEFAULT: xlim = XLIM_DEFAULT
    else: xlim = (np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    print 'XLIM',xlim,'YLIM',ylim
    for ax_index, img in enumerate([adcimg, labelimg, bkgimg, showerimg, trackimg]):

        ax = ax_arr[ax_index]
        ax.imshow(img,cmap='jet',interpolation='none',vmin=0,vmax=255,origin='lower')
        #ax.set_aspect(ASPECT)
        ax.set_ylim(*ylim)
        ax.set_xlim(*xlim)
        ax.set_ylabel('Time [6 ticks]',fontsize=20)
        ax.set_xlabel('Wire',fontsize=20)
        ax.tick_params(labelsize=20)
    
    #plt.colorbar()
    plt.show()
    

In [ ]:
ASPECT=1.0
#135 x 100
ENTRIES={  1: [(200,358),(382,501)],
           3: [(322,368),(456,486)],
           4: [(211,370),(385,508)],
           9: [(292,370),(259,351)],
          12: [(296,369),(368,421)],
          17: [( 21,245),(237,490)],
          18: [(132,299),(257,484)],
          20: [(282,369),(354,413)],
          27: [(329,369),(383,440)],
          29: [( 50,227),(237,322)],
          30: [(116,175),(246,296)],
          32: [(210,366),(237,300)],
          38: [( 19,235),(297,398)],
          41: [( 55,240),(359,508)],
          45: [( 94,316),(237,325)],
          52: [(278,366),(294,370)],
          56: [( 91,345),(237,357)],
          58: [(108,343),(245,396)],
          63: [(317,366),(450,497)],
          68: [(265,367),(345,429)],
          77: [(133,203),(425,501)],
          79: [( 74,360),(237,322)],
          81: [(310,369),(373,407)],
          82: [(213,369),(237,316)],
          89: [(226,370),(348,393)],
          90: [(198,358),(349,383)],
          92: [(  3,253),(373,508)],
          93: [(233,370),(358,477)],
          94: [(111,345),(245,311)],
          95: [(116,366),(373,490)],
          96: [( 13,364),(200,400)],
          98: [( 58,239),(294,478)],
          99: [(125,260),(242,342)]
         }

keys=ENTRIES.keys()
keys.sort()
for entry in keys:
    xlim,ylim=ENTRIES[entry]
    print 'Entry',entry
    ch.GetEntry(entry)
#    xlim=xlim_arr[index]
#    ylim=ylim_arr[index]

    image2d_v = ch.image2d_ssnet_plane2_branch.Image2DArray()
    
    bkgimg    = np.flipud(larcv.as_ndarray(image2d_v[0]).astype(np.float32))#*255.
    showerimg = np.flipud(larcv.as_ndarray(image2d_v[1]).astype(np.float32))#*255.
    trackimg  = np.flipud(larcv.as_ndarray(image2d_v[2]).astype(np.float32))#*255.
    adcimg    = np.flipud(larcv.as_ndarray(image2d_v[3]).astype(np.float32))#*1.
    labelimg  = np.flipud(larcv.as_ndarray(image2d_v[4]).astype(np.float32))#*80.


    ssnetimg  = (((showerimg) > 0.5) * 80) + ((trackimg > 0.5) * 160)

    bkgimg    *= 255.
    showerimg *= 255.
    trackimg  *= 255.
    labelimg  *= 80.

    #
    # adc + labels
    #
    mat_arr = [adcimg, labelimg, ssnetimg]
    img_arr = []

    fig,ax_arr=plt.subplots(1,len(mat_arr),figsize=(48,12),facecolor='w')
    for ax_index, mat in enumerate(mat_arr):

        ax = ax_arr[ax_index]
        img = ax.imshow(mat,cmap='jet',interpolation='none',vmin=0,vmax=255,origin='lower')
        img_arr.append(img)
        #ax.set_aspect(ASPECT)
        ax.set_ylim(*ylim)
        ax.set_xlim(*xlim)
        ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
        ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
        ax.tick_params(labelsize=20)

    fig.subplots_adjust(right=0.82)
    cbar_ax = fig.add_axes([0.85, 0.2, 0.01, 0.65])
    fig.colorbar(img_arr[0], cax=cbar_ax)
    #fig.colorbar(img_arr[0])
    plt.show()
    
    #
    # adc + score
    #
    
    mat_arr = [adcimg, showerimg, trackimg]
    img_arr = []

    fig,ax_arr=plt.subplots(1,len(mat_arr),figsize=(48,12),facecolor='w')
    for ax_index, mat in enumerate(mat_arr):

        ax = ax_arr[ax_index]
        img = ax.imshow(mat,cmap='jet',interpolation='none',vmin=0,vmax=255,origin='lower')
        img_arr.append(img)
        #ax.set_aspect(ASPECT)
        ax.set_ylim(*ylim)
        ax.set_xlim(*xlim)
        ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
        ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
        ax.tick_params(labelsize=20)

    fig.subplots_adjust(right=0.82)
    cbar_ax = fig.add_axes([0.85, 0.2, 0.01, 0.65])
    fig.colorbar(img_arr[0], cax=cbar_ax)
    #fig.colorbar(img_arr[0])
    plt.show()

In [ ]:
ASPECT=1.0

#ENTRY=99;xlim=(124,259);ylim=(242,342)

ENTRY=32;xlim=(240,366);ylim=(240,280) 


print 'Entry',ENTRY

ch.GetEntry(ENTRY)

image2d_v = ch.image2d_ssnet_plane2_branch.Image2DArray()

print image2d_v[0].meta().dump()
    
bkgimg    = np.flipud(larcv.as_ndarray(image2d_v[0]).astype(np.float32))#*255.
showerimg = np.flipud(larcv.as_ndarray(image2d_v[1]).astype(np.float32))#*255.
trackimg  = np.flipud(larcv.as_ndarray(image2d_v[2]).astype(np.float32))#*255.
adcimg    = np.flipud(larcv.as_ndarray(image2d_v[3]).astype(np.float32))#*1.
labelimg  = np.flipud(larcv.as_ndarray(image2d_v[4]).astype(np.float32))#*80.


ssnetimg  = (((showerimg) > 0.5) * 80.) + ((trackimg > 0.5) * 160.)

#bkgimg    *= 255.
#showerimg *= 255.
#trackimg  *= 255.
labelimg  *= 80.
#ssnetimg  *= 80.

Scale=[255,255,1,1,1,255]

#for ax_index, img in enumerate([adcimg, labelimg, bkgimg, showerimg, trackimg, ssnetimg]):

fig,ax=plt.subplots(figsize=(13,8),facecolor='w')
mat=plt.imshow(adcimg,cmap='jet',interpolation='none',vmin=0,vmax=255.,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.335, 0.02, 0.358])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(20,260,40))
plt.show()

fig,ax=plt.subplots(figsize=(13,8),facecolor='w')
mat=plt.imshow(labelimg,cmap='jet',interpolation='none',vmin=0,vmax=255.,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.335, 0.02, 0.358])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(20,260,40))
plt.show()

fig,ax=plt.subplots(figsize=(13,8),facecolor='w')
mat=ax.imshow(bkgimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.335, 0.02, 0.358])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(0,1.2,0.2))
plt.show()

fig,ax=plt.subplots(figsize=(13,8),facecolor='w')
mat=ax.imshow(showerimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.335, 0.02, 0.358])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(0,1.2,0.2))
plt.show()

fig,ax=plt.subplots(figsize=(13,8),facecolor='w')
mat=ax.imshow(trackimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.335, 0.02, 0.358])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(0,1.2,0.2))
plt.show()

fig,ax_arr=plt.subplots(3,1,figsize=(13,13),facecolor='w')
ax=ax_arr[0]
mat=ax.imshow(bkgimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax=ax_arr[1]
mat=ax.imshow(showerimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax=ax_arr[2]
mat=ax.imshow(trackimg,cmap='jet',interpolation='none',vmin=0,vmax=1,origin='lower')
ax.set_aspect(1.)
ax.set_ylim(*ylim)
ax.set_xlim(*xlim)
#ax.set_ylabel('Time',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.set_xlabel('Wire',fontsize=24,fontname='Georgia',fontweight='bold')
#ax.tick_params(labelsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.84, 0.127, 0.02, 0.772])
cbar_ax.tick_params(labelsize=20)
cbar=fig.colorbar(mat, cax=cbar_ax,ticks=np.arange(0,1.1,0.1))
plt.show()